<h2 style="font-family:verdana;font-weight:bold; text-align: center;">RSNA-MICCAI Brain Tumor Radiogenomic Classification</h2>
<h3 style="text-align: center; font-family:courier; font-weight:bold;">Predict the status of a genetic biomarker important for brain cancer treatment</h3>

<img src="https://faculty.mdanderson.org/content/dam/mdanderson/images/publications/cancerwise/Graphics/Braintumor.jpg.resize.jpg" height="400" width="400" style="display: block;margin-left: auto;margin-right: auto;"> 

<h2 style = "font-family:verdana;font-weight:bold; text-align: center;"> About RSNA </h2>

<img src="https://www.rsna.org/images/rsna/home/line_r.svg" width="100" style="display: block;margin-left: auto;margin-right: auto;">
<p style = "font-family:verdana;"> <b>The Radiological Society of North America</b> (<a href = "https://www.rsna.org/"> RSNA </a>) is an international society of radiologists, medical physicists and other medical professionals with more than 54,000 members across the globe. Based in Oak Brook, Illinois, RSNA was established in 1915. </p>
    
<p style = "font-family:verdana;"> RSNA hosts one of the world's largest annual medical meetings, a week-long event which starts the last Sunday of November in Chicago. The society also publishes two peer-reviewed journals: Radiology, offering radiology research and reviews, and RadioGraphics, dedicated to continuing education in radiology. </p>
    
<p style = "font-family:verdana;">The Society also develops and offers informatics-based software solutions in support of a universal electronic health record, sponsors research to advance quantitative imaging biomarkers, and conducts outreach to enhance education in developing nations. Through its Research and Education Foundation, RSNA provides millions of dollars in funding to investigators. </p>

In [ ]:
#------------------------------------------------
##install gdcm for DICOM image data preprocessing
#------------------------------------------------

!conda install -c conda-forge gdcm -y

In [ ]:
#---------------------------
##import necessary libraries
#---------------------------

import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np

import cv2
import matplotlib.pyplot as plt
from matplotlib import  rc
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import PIL
from PIL import Image

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import warnings
import os
warnings.filterwarnings('ignore')

In [ ]:
TRAIN_PATH = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
TEST_PATH = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

HEIGHT,WIDTH = 512,512

#----------------------------
##Function to obtain metadata
#----------------------------
def get_metadata(mainpath):
    cases = os.listdir(mainpath)
    flair,t1w,t1wce,t2w = [],[],[],[]
    for case in cases:
        casepath = os.path.join(mainpath,case)
        flairpath = os.path.join(casepath,"FLAIR")
        t1wpath = os.path.join(casepath,"T1w")
        t1wcepath = os.path.join(casepath,"T1wCE")
        t2wpath = os.path.join(casepath,"T2w")
        
        flair.append(len(os.listdir(flairpath)))
        t1w.append(len(os.listdir(t1wpath)))
        t1wce.append(len(os.listdir(t1wcepath)))
        t2w.append(len(os.listdir(t2wpath)))
    df = pd.DataFrame.from_dict({
        "CASE ID" : cases,
        "FLAIR" : flair,
        "T1w" : t1w,
        "T1wCE" : t1wce,
        "T2w" : t2w
    })
    return df

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">Information about Training Images</h2>

## Dataset Structure

- train / test
   - case [5 digited number]
       - FLAIR
          - Image-1.dicom
          - ....
       - T1w
          - Image-1.dicom
          - ....
       - T1wCE
          - Image-1.dicom
          - ....
       - T2w
          - Image-1.dicom
          - ....
   - ....          
- train_labels.csv
- sample_submission.csv

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">Pandas Profiling</h2>

<img src="https://pandas-profiling.github.io/pandas-profiling/docs/assets/logo_header.png" style= "display: block; margin-left: auto; margin-right: auto;" height="300" width="300">

<p>Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. Besides, if this is not enough to convince us to use this tool, it also generates interactive reports in web format that can be presented to any person, even if they don’t know programming.</p>
<a style = "text-align :center; font-family:verdana; color:blue; font-size:25px" href = "https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/"> Know more about Pandas Profiling </a>

In [ ]:
#----------------------
##Get Training Metadata
#----------------------

train_df = get_metadata(TRAIN_PATH)
train_df.head(3)

In [ ]:
#-------------
##Train report
#-------------

train_report = ProfileReport(train_df,title="Metadata of Training images")
train_report.to_file("./train_metadata.html")
train_report

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">Information about Training Labels</h2>

In [ ]:
label_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
label_df.head(3)

In [ ]:
#--------------------
##Train Labels report
#--------------------

labels_report = ProfileReport(label_df,title="Training labels")
labels_report.to_file("./train_labels.html")
labels_report

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">Information about Test Images</h2>

In [ ]:
#------------------
##Get Test Metadata
#------------------

test_df = get_metadata(TEST_PATH)
test_df.head(3)

In [ ]:
#------------
##Test report
#------------

test_report = ProfileReport(test_df,title="Metadata of Test images")
test_report.to_file("./test_metadata.html")
test_report

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">DICOM to Array</h2>

<h2 style="font-weight:'bold'; color:red; font-family:verdana; text-align: center;">What is DICOM format</h2>
<p> <b>Digital Imaging and Communications in Medicine</b> (DICOM) is the standard for the communication and management of medical imaging information and related data. DICOM is most commonly used for storing and transmitting medical images enabling the integration of medical imaging devices such as scanners, servers, workstations, printers, network hardware.</p>

<a href = "https://en.wikipedia.org/wiki/DICOM" style="font-weight:'bold'; color:blue; font-family:monospace; text-align: center;"> Read more about DICOM here</a>

<h2 style="font-weight:'bold'; color:red; font-family:verdana; text-align: center;">Why is DICOM used</h2>

<p>Following are some of applications of using DICOM format in medical imaging</p>

- Collaboration With Existing IT Systems
- High-Performance Review
- Complete Scanning and Image Reviewing

<a href = "https://www.covetus.com/blog/how-is-dicom-important-beneficial-for-the-healthcare-industry" style="font-weight:'bold'; color:blue; font-family:monospace; text-align: center;"> Read more about DICOM applications here</a>

In [ ]:
##---------------------------------------
#DICOM to pixel array converting function
##---------------------------------------

# Ref : https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
def dicom2arr(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    if voi_lut:
        arr = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        arr = dicom.pixel_array
               
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        arr = np.amax(arr) - arr
        
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    arr = (arr * 255).astype(np.uint8)
    
    arr = cv2.resize(arr,(HEIGHT,WIDTH))
    return arr

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">MRIs Visualization</h2>

<h2 style="font-weight:'bold'; color:red; font-family:verdana; text-align: center;">What is MRI </h2>

<p> <b>Medical Resonance Imaging</b> (MRI) is a medical imaging technique used in radiology to form pictures of the anatomy and the physiological processes of the body. MRI scanners use strong magnetic fields, magnetic field gradients, and radio waves to generate images of the organs in the body. MRI does not involve X-rays or the use of ionizing radiation, which distinguishes it from <b>CT</b> and <b>PET</b> scans. MRI is a medical application of <b>Nuclear Magnetic Resonance (NMR)</b> which can also be used for imaging in other NMR applications, such as NMR spectroscopy.</p>

<h2 style="font-weight:'bold'; color:red; font-family:verdana; text-align: center;">Types of MRI Scans </h2>

- **Fluid Attenuated Inversion Recovery (FLAIR)**

                      Fluid attenuated inversion recovery (FLAIR) is a special inversion recovery sequence with a long inversion time. This removes signal from the cerebrospinal fluid in the resulting images 1. Brain tissue on FLAIR images appears similar to T2 weighted images with grey matter brighter than white matter but CSF is dark instead of bright.
                      
                      
- **T1-weighted (T1)**
                      T1-weighted images are produced by using short TE and TR times. The contrast and brightness of the image are predominately determined by T1 properties of tissue.
    - **T1-weighted pre-contrast (T1w)**
    - **T1-weighted post-contrast (T1Gd)**
    
    
- **T2-weighted (T2)**
    
                     T2 weighted image (T2WI) is one of the basic pulse sequences on MRI. The sequence weighting highlights differences on the T2 relaxation time of tissues.
                     
                     
                     
 <a href = " https://radiopaedia.org/?lang=gb" style="font-weight:'bold'; color:blue; font-family:monospace; text-align: center;"> Read more about MRI Scanning here</a>

 <a href = "https://case.edu/med/neurology/NR/MRI%20Basics.htm" style="font-weight:'bold'; color:blue; font-family:monospace; text-align: center;"> For quick learning check this.</a>

In [ ]:
#----------------------
##sample images display
#----------------------

flair = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-109.dcm"
t1w = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-19.dcm"
t1wce = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-50.dcm"
t2w = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/Image-180.dcm"

flair_arr = dicom2arr(flair)
t1w_arr = dicom2arr(t1w)
t1wce_arr = dicom2arr(t1wce)
t2w_arr = dicom2arr(t2w)

img_ls = [flair_arr,t1w_arr,t1wce_arr,t2w_arr]
title_ls = ["FLAIR","T1-weighted pre contrast", "T1-weighted post contrast", "T2-weighted"]

fig,ax = plt.subplots(nrows=1,ncols=4,figsize=(20,10))
for i in range(4):
    ax[i].imshow(img_ls[i],cmap="viridis")
    ax[i].axis("off")
    ax[i].set_title(title_ls[i])
    
plt.show()

<h2 style = "text-align :center; font-family:verdana; color:black; background-image: url(https://image.freepik.com/free-vector/gradient-background-green-shades_23-2148363157.jpg); ">Animation of MRIs</h2>

In [ ]:
#--------------------------
##Aniamating FLAIR type MRI
#--------------------------
flair = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR"
name_ls = os.listdir(flair)
path_ls = [os.path.join(flair,i) for i in name_ls]

img_ls = [dicom2arr(path) for path in path_ls]

In [ ]:
rc('animation', html='jshtml') 

def get_animation(img_ls):
    fig = plt.figure(figsize=(8,8))
    im_arr = plt.imshow(img_ls[0],cmap="viridis")
    plt.axis("off")
    plt.title("FLAIR")
    def update(i):
        im_arr.set_array(img_ls[i])
        return [im_arr]
    anim  = animation.FuncAnimation(fig,update,frames=len(img_ls),interval=50)
    anim.save("./FLAIR.mp4")
    return anim

get_animation(img_ls)

<img src="https://t4.ftcdn.net/jpg/03/54/26/09/360_F_354260981_mvf4Yt39tO1iAWkXeFcPayv0OkTw6p4j.jpg" style= "display: block;margin-left: auto; margin-right: auto;" height="100" width="300">

<h1 style = "text-align :center; color:white; background-image: url(https://cdn.pixabay.com/photo/2014/06/16/23/40/blue-370128__340.png); ">Do upvote if you found this notebook useful🙂</h1>